In [2]:
from data_prep_utils import *
from plot_prep_utils import *
import pandas as pd

dfe = get_eigenphi_march_blocks_with_to_and_from()
dft = get_titan_march_blocks_with_to_and_from()

In [5]:
print(len(dfe["to_address"].unique()))
print(len(dfe["from_address"].unique()))
print(len(dft["to_address"].unique()))
print(len(dft["from_address"].unique()))



5842
7802
1682447
2760243


In [8]:


def get_top_rewards(df, address_type, output_file):
    """
    Processes rewards data to return the top 50 addresses and optionally saves it to a CSV file.
    
    Args:
    df (pd.DataFrame): DataFrame containing the transaction data.
    address_type (str): 'to_address' or 'from_address' for reward aggregation.
    
    Returns:
    pd.DataFrame: DataFrame holding the top 50 addresses sorted by rewards.
    """
    if address_type not in ['to_address', 'from_address']:
        raise ValueError("address_type must be 'to_address' or 'from_address'")

    subtotal_df = df.groupby(address_type)["builder_reward"].sum().reset_index()
    subtotal_df["builder_reward"] = subtotal_df["builder_reward"].astype(float)
    sorted_df = subtotal_df.sort_values("builder_reward", ascending=False).head(50)

    if output_file:
        sorted_df.to_csv(output_file, index=False)
        print(f"DataFrame exported to {output_file}")

    return sorted_df


In [9]:
top50_subtotal_from_dft = get_top_rewards(dft, 'from_address', 'top_50_titan_rewards_from.csv')
top50_subtotal_to_dft = get_top_rewards(dft, 'to_address', 'top_50_titan_rewards_to.csv')
top50_subtotal_from_dfe = get_top_rewards(dfe, 'from_address', 'top_50_eigenphi_rewards_from.csv')   
top50_subtotal_to_dfe = get_top_rewards(dfe, 'to_address', 'top_50_eigenphi_rewards_to.csv')


DataFrame exported to top_50_titan_rewards_from.csv
DataFrame exported to top_50_titan_rewards_to.csv
DataFrame exported to top_50_eigenphi_rewards_from.csv
DataFrame exported to top_50_eigenphi_rewards_to.csv


In [10]:
import pandas as pd

def find_common_addresses(df1, df2, address_column, output_file):
    """
    Finds common addresses between two datasets and saves the result to a CSV file.

    Args:
        df1 (pd.DataFrame): First dataset containing addresses.
        df2 (pd.DataFrame): Second dataset containing addresses.
        address_column (str): The name of the column to merge on ('from_address' or 'to_address').
        output_file (str): The filename for the output CSV.
    """
    # Merging on specified address column
    common_addresses = pd.merge(df1, df2, on=address_column)
    print(f"Common {address_column} between datasets:")
    # print(common_addresses)

    # Save to CSV
    common_addresses.to_csv(output_file, index=False)
    # print(f"DataFrame exported to {output_file}")

# Assuming subtotal_from_dft, subtotal_from_dfe, subtotal_to_dft, and subtotal_to_dfe are already defined:

# Find common from_addresses between EigenPhi and Titan
find_common_addresses(top50_subtotal_from_dft, top50_subtotal_from_dfe, 'from_address', 'common_from_addresses.csv')

# Find common to_addresses between EigenPhi and Titan
find_common_addresses(top50_subtotal_to_dft, top50_subtotal_to_dfe, 'to_address', 'common_to_addresses.csv')


Common from_address between datasets:
Common to_address between datasets:


In [15]:
# find uncommon addresses between EigenPhi and Titan
import pandas as pd

def find_uncommon_addresses(df1, df2, address_column, eigenphi_output_file, titan_output_file):
    """
    Finds uncommon addresses between two datasets and saves the result to a CSV file.

    Args:
        df1 (pd.DataFrame): First dataset containing addresses.
        df2 (pd.DataFrame): Second dataset containing addresses.
        address_column (str): The name of the column to merge on ('from_address' or 'to_address').
        output_file (str): The filename for the output CSV.
    """
    # Assuming subtotal_from_dft and subtotal_from_dfe are already defined DataFrame from previous steps:
    # Perform an outer join merge
    all_addresses = pd.merge(df1, df2, on=address_column, how='outer', indicator=True)
    
    # Filter results to get non-overlapping addresses
    uncommon_addresses = all_addresses[all_addresses['_merge'] != 'both']
    
    
    # pd.merge adds a column '_merge' that indicates from which DataFrame the rows come from:
    # "left_only" for EigenPhi, "right_only" for Titan, and "both" for common.

    # Optionally, split into two DataFrames depending on source
    uncommon_eigenphi = uncommon_addresses[uncommon_addresses['_merge'] == 'left_only']
    uncommon_titan = uncommon_addresses[uncommon_addresses['_merge'] == 'right_only']

    uncommon_eigenphi[[address_column]].to_csv(eigenphi_output_file, index=False)
    uncommon_titan[[address_column]].to_csv(titan_output_file, index=False)
    
    
# Find uncommon from_addresses between EigenPhi and Titan
find_uncommon_addresses(top50_subtotal_from_dfe, top50_subtotal_from_dft, 'from_address', 'uncommon_from_addresses_eigenphi.csv', 'uncommon_from_addresses_titan.csv')


# Find uncommon to_addresses between EigenPhi and Titan
find_uncommon_addresses(top50_subtotal_to_dfe, top50_subtotal_to_dft, 'to_address', 'uncommon_to_addresses_eigenphi.csv', 'uncommon_to_addresses_titan.csv')


KeyError: 'to_address'